# Some interactive charts

> Once upon a midnight dreary

In [ ]:
# from bokeh.sampledata.periodic_table import elements
import altair as alt
import pandas as pd
import panel as pn
import param

In [ ]:
pn.extension('vega')

## Panel Widget
Must use `widget.embed()`.

In [ ]:
slider = pn.widgets.IntSlider(start=0, end=10)

@pn.depends(slider.param.value)
def callback(value):
    return '%d * 5 = %d' % (value, value*5)

widget1 = pn.Row(slider, callback)
widget1

Row
    [0] IntSlider(end=10)
    [1] ParamFunction(function)

In [ ]:
widget1.embed(states={slider: [i for i in range(11)]})

## Panel & Altair

In [ ]:
elements = pd.read_csv('https://raw.githubusercontent.com/bokeh/bokeh/aae3034a9df90119c3c8c71ddf7a15e41821a5d0/src/bokeh/sampledata/_data/elements.csv')
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2


In [ ]:
class PeriodSelector(param.Parameterized):
    slider = param.Integer(6, bounds=(1, 6), step=1)
    
    @param.depends('slider', watch=True)
    def _update_data(self):
        return elements.query(f"period == {self.slider}").copy()
    
    def view(self):
        data = self._update_data()
        chart = (
            alt.Chart(data, title=f"period {self.slider} elements")
            .mark_point()
            .encode(
                x=alt.X('symbol:N', sort=data.symbol.to_numpy()),
                y='electronegativity',
                tooltip=['symbol','atomic mass']
            )
            .properties(width=400, height=200)
        )
        return chart

period_selector = PeriodSelector(name='Periodic Trends in Electronegativity')
widget2 = pn.Column(period_selector.param, period_selector.view)
widget2

Column
    [0] Column(margin=5, name='Periodic Trends i..., width=300)
        [0] StaticText(value='<b>Periodic T...)
        [1] IntSlider(end=6, name='Slider', start=1, value=6)
    [1] ParamMethod(method)

In [ ]:
widget2.embed(states={widget2[0][1]: list(range(1, 7))})